<a href="https://colab.research.google.com/github/kumpaten/masters-thesis-code/blob/main/USPTO_patents_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The SQL query used to retrieve the raw patent data found in "/content/drive/MyDrive/bq-results-20250212-194745-1739389687161/bq-results-20250212-194745-1739389687161.csv" in the next step
"""
SELECT
    p.date,
    p.id,
    p.num_claims,
    a.assignee_id,
    a.type,
    a.organization
  FROM
    `patents-public-data.patentsview.patent` AS p
  JOIN
    `patents-public-data.patentsview.rawassignee` AS a
      ON p.id = a.patent_id
  WHERE
    p.date BETWEEN '2008-01-01' AND '2021-12-31'
"""

In [1]:
#@title Filter Organization Names for Each Company and Aggregate num_claims by Year

import re
import pandas as pd
from google.colab import drive

# --- Step 1: Mount Google Drive and load the CSV file ---
drive.mount('/content/drive')

# Update the file path as necessary
file_path = '/content/drive/MyDrive/bq-results-20250212-194745-1739389687161/bq-results-20250212-194745-1739389687161.csv'
df = pd.read_csv(file_path)

# Ensure required columns exist: "organization", "patent_date", "num_claims"
for col in ["organization", "date", "num_claims"]:
    if col not in df.columns:
        raise ValueError(f"The CSV file must have a '{col}' column.")

# Ensure all organization values are strings (fill missing values with empty string)
df["organization"] = df["organization"].fillna("").astype(str)

# Convert "patent_date" to datetime and extract the year
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year

# --- Step 2: Define filtering rules for each company ---
# For each company we define:
#   - "allowed": a list of keywords that must be present (case-insensitive)
#   - "forbidden": a list of keywords that if present (case-insensitive) indicate the name is not genuine
company_filters = {
    "microsoft": {
         "allowed": ["microsoft"],
         "forbidden": ["orthopedics", "hitachi", "tangis", "systems inc."]
    },
    "apple": {
         "allowed": ["apple"],
         "forbidden": [ "applera",         # different company
                        "appleton",        # e.g., Appleton Papers, Appleton Grp, etc.
                        "snapple",         # beverage companies
                        "dapple",          # Dapple Equine Products
                        "kappler",         # Kappler-related names
                        "grapplers",       # Grapplers, Inc.
                        "three apples",    # Three Apples Cosmetics
                        "zapple",          # ZAPPLE, INC.
                        "pineapple",       # Project Pineapple, Pineapple Express
                        "applejack",       # Applejack 199 L.P.
                        "inferno",         # Apple Inferno, Inc.
                        "biomedical",      # Apple Biomedical*
                        "improvement",     # Midwest Apple Improvement Association
                        "project pineapple",  # Project Pineapple, LLC
                        "shanghai",        # Shanghai Apple Flavor & Fragrance Co.
                        "ju studio",       # Apple Ju Studio Inc.
                        "applegate",       # Applegate Livestock Equipment, Inc.
                        "partners",        # Apple Partners, LP
                        "applesauce",      # Applesauce Project Inc.
                        "little green",    # Little Green Apples, Inc.
                        "wapple",          # Wapple.net Ltd
                        "applexion",       # Applexion
                        "electric car",    # Apple Electric Car, Inc.
                        "ii 'c",          # APPLE II 'C.
                        "appled",         # Appled Materials, Inc.
                        "apples-to-go",
                        "appleid materials, inc",
                        "appleboy",
                        "mayapple",
                        "candi"]
    },
    "salesforce": {
         "allowed": ["salesforce"],
         "forbidden": []
    },
    "facebook": {
         "allowed": ["facebook"],
         "forbidden": []
    },
    # For SAP, we will later use a stricter (whitelist) approach.
    "sap": {
         "allowed": ["sap"],
         "forbidden": ["sapphire",       # e.g., Sapphire Energy, etc.
                       "sapporo",        # e.g., Sapporo Breweries, etc.
                       "saplo",          # e.g., Saplo AB
                       "sapling",        # e.g., Sapling Company, The Sapling Company
                       "aisapack",       # e.g., AISAPACK Holding S.A.
                       "asap",           # e.g., ASAP BreatheAssist, etc.
                       "sapiens",        # e.g., Sapiens Steering Brain Stimulation, etc.
                       "saphinov",       # e.g., Saphinov S.N.C.
                       "extrus",         # covers "extrusions", "extrustions", etc.
                       "sapere",         # e.g., Sapere IP, LLC
                       "sapurast",       # e.g., Sapurast Research LLC
                       "portals",        # e.g., SAP Portals Israel Ltd, etc.
                       "ohio aerosapce",  # Ohio Aerosapce Institute
                       "profiler",       # e.g., Sapa Profiler AB
                       "sapdesign",      # e.g., SAPDesign AS
                       "sapo u.s.a",     # Sapo U.S.A. Corp.
                       "fricaeco",       # e.g., Fricaeco America SAPI de C.V.
                       "sapient",        # e.g., Sapience Corporation
                       "sapnsion",       # e.g., Sapnsion LLC
                       "saphran",        # e.g., Saphran Inc. / Saphran, Inc.
                       "sapiotec",       # e.g., SAPIOTEC GMBH
                       "saphena",        # e.g., Saphena Medical, Inc.
                       "sapheon",        # e.g., Sapheon, Inc.
                       "sapag",          # e.g., SAPAG
                       "sap link",       # e.g., Sap Link Technology Corp.
                       "sapsa",          # e.g., Sapsa Bedding S.R.L.
                       "sapturf",        # e.g., Sapturf, LLC
                       "sapre",         # e.g., SAPREX, LLC
                       "sappel",         # e.g., Sappel
                       "saponaqua",      # e.g., Saponaqua International Limited
                       "drinksapor",     # e.g., DRINK SAPORÉ INC.
                       "sap markets",    # e.g., Sapmarkets Inc.
                       "mesaplex",       # e.g., Mesaplexx Pty Ltd, etc.
                       "isapac",         # e.g., Isapac Participaçöes SA.
                       "universita",     # academic institutions
                       "sap link",
                       "whatsapp",
                       "sapience", "yehuda", "sensaphonics", "heat", "sappi", "sapna", "sapa", "sapir", "guisapet", "saphinon", "drossapharm", "sapheco", "ursapharm", "konstantin", "saphire", "sensapex", "sapio", "sapir", "sapience", "sapienza", "visapa", "sapmarkets", "arisaph", "chesapeake", "crimsape", "misapplied", "sapiselco", "drink", "sensapex", "amsapplied", "sapp", "saprise"]
    },
    "google": {
         "allowed": ["google"],
         "forbidden": []
    },
    "oracle": {
         "allowed": ["oracle"],
         "forbidden": []
    },
    "cisco": {
         "allowed": ["cisco"],
         "forbidden": ["san francisco", "university", "fundacao", "certis", "photonics", "aldef global"]
    },
    "accenture": {
         "allowed": ["accenture"],
         "forbidden": ["properties", " corporation", " ans"]
    },
    "ibm": {
         "allowed": ["ibm", "international business machine", "international-business", "i.b.m", "ibm corp"],
         "forbidden": ["the libman", "instituto", "institute", "ibmt", "ibmv", "hibm", "ibmc"]
    }
}

# --- Step 3: Filter organization names for each company ---
results_dict = {}

for company, rules in company_filters.items():

    allowed_keywords = rules["allowed"]
    forbidden_keywords = rules["forbidden"]
    condition_allowed = df["organization"].str.lower().apply(
        lambda org: any(keyword in org for keyword in allowed_keywords)
    )
    condition_forbidden = df["organization"].str.lower().apply(
        lambda org: any(keyword in org for keyword in forbidden_keywords)
    )
    filtered_orgs = df[condition_allowed & (~condition_forbidden)]["organization"].unique().tolist()

    results_dict[company] = filtered_orgs

# --- Step 4: Aggregate num_claims by year for each company ---
agg_results = {}

for company, org_list in results_dict.items():
    # Filter the DataFrame to rows where "organization" is in the filtered list for this company.
    subset = df[df["organization"].isin(org_list)]
    # Group by "year" and sum "num_claims"
    grouped = subset.groupby("year")["num_claims"].sum().reset_index()
    grouped["company"] = company  # add a company column for later pivoting
    agg_results[company] = grouped

# Combine all companies' aggregated results into one DataFrame.
agg_df = pd.concat(agg_results.values(), ignore_index=True)

# Create a pivot table: rows = year, columns = company, values = sum(num_claims)
pivot_table = agg_df.pivot(index="year", columns="company", values="num_claims").fillna(0)

# --- Step 5: Print the pivot table ---
print("Pivot Table: Aggregated num_claims by Year for Each Company")
print(pivot_table)


Mounted at /content/drive
Pivot Table: Aggregated num_claims by Year for Each Company
company  accenture    apple    cisco  facebook   google       ibm  microsoft  \
year                                                                           
2008        1680.0   5798.0  19911.0       0.0   1650.0   47208.0    45413.0   
2009        1107.0   8536.0  23975.0       0.0   3619.0   56806.0    58555.0   
2010        2575.0  16090.0  27774.0     143.0   7317.0   91547.0    58039.0   
2011        3011.0  17369.0  22907.0     315.0  11265.0  100387.0    42690.0   
2012        2440.0  27101.0  20947.0     991.0  26281.0  107610.0    47428.0   
2013        3148.0  40565.0  18412.0    2723.0  41346.0  113561.0    50194.0   
2014        3726.0  44339.0  22184.0    6551.0  55049.0  122049.0    54108.0   
2015        2510.0  41751.0  19267.0    8206.0  64758.0  112817.0    47213.0   
2016        3159.0  44615.0  19070.0    9107.0  63891.0  112859.0    47479.0   
2017        2964.0  48413.0  18744

In [ ]:
# --- Step 6: Save the pivot table to CSV and download it ---

# Define output file path on your Google Drive
output_csv_path = '/content/drive/MyDrive/aggregated_claims_by_year.csv'
pivot_table.to_csv(output_csv_path)
print("\nCSV file saved to:", output_csv_path)

#(Optional) Download the CSV file to your local machine
#from google.colab import files
#files.download(output_csv_path)


CSV file saved to: /content/drive/MyDrive/aggregated_claims_by_year.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>